In [ ]:
0、id、int行の通し番号
1、age、int、年齢
2、job、varchar、職種
3、marital、varchar、未婚/既婚
4、education、varchar、教育水準
5、default、varchar、債務不履行があるか（yes, no）
6、balance、int、年間平均残高（€）
7、housing、varchar、住宅ローン（yes, no）
8、loan、varchar、個人ローン（yes, no）
9、contact、varchar、連絡方法
10、dayint、最終接触日
11、monthchar、最終接触月
12、duration、int、最終接触時間（秒）
13、compaign、int、現キャンペーンにおける接触回数
14、pdaysint、経過日数：前キャンペーン接触後の日数
15、previous、int、接触実績：現キャンペーン以前までに顧客に接触した回数
16、poutcome、varchar、前回のキャンペーンの成果
17、y、boolean、定額預金申し込み有無（1:有り, 0:無し）
'''

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [25]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
sample = pd.read_csv("./data/submit_sample.csv", header=None)

In [26]:
print(train.shape)
print(test.shape)
print(sample.shape)

(27128, 18)
(18083, 17)
(18083, 2)


In [27]:
train.isnull().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [28]:
test.isnull().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
dtype: int64

In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27128 entries, 0 to 27127
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         27128 non-null  int64 
 1   age        27128 non-null  int64 
 2   job        27128 non-null  object
 3   marital    27128 non-null  object
 4   education  27128 non-null  object
 5   default    27128 non-null  object
 6   balance    27128 non-null  int64 
 7   housing    27128 non-null  object
 8   loan       27128 non-null  object
 9   contact    27128 non-null  object
 10  day        27128 non-null  int64 
 11  month      27128 non-null  object
 12  duration   27128 non-null  int64 
 13  campaign   27128 non-null  int64 
 14  pdays      27128 non-null  int64 
 15  previous   27128 non-null  int64 
 16  poutcome   27128 non-null  object
 17  y          27128 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.7+ MB


In [30]:
train.columns

Index(['id', 'age', 'job', 'marital', 'education', 'default', 'balance',
       'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'y'],
      dtype='object')

In [31]:
#カテゴリ変数として扱いたい
#job, marital, education, default, housing, loan, contact, month, poutcome
for col in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]:
    train[col] = train[col].astype("category")

In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27128 entries, 0 to 27127
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         27128 non-null  int64   
 1   age        27128 non-null  int64   
 2   job        27128 non-null  category
 3   marital    27128 non-null  category
 4   education  27128 non-null  category
 5   default    27128 non-null  category
 6   balance    27128 non-null  int64   
 7   housing    27128 non-null  category
 8   loan       27128 non-null  category
 9   contact    27128 non-null  category
 10  day        27128 non-null  int64   
 11  month      27128 non-null  category
 12  duration   27128 non-null  int64   
 13  campaign   27128 non-null  int64   
 14  pdays      27128 non-null  int64   
 15  previous   27128 non-null  int64   
 16  poutcome   27128 non-null  category
 17  y          27128 non-null  int64   
dtypes: category(9), int64(9)
memory usage: 2.1 MB


In [67]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
import lightgbm as lgb

df_train, df_val = train_test_split(train, test_size=0.2, random_state=0)

col = "y"
trainy = df_train[col]
trainX = df_train.drop(col, axis=1)

valy = df_val[col]
valX = df_val.drop(col, axis=1)

trains = lgb.Dataset(trainX, trainy)
valids = lgb.Dataset(valX, valy)

params = {
    "objective":"binary",
    "metrics":"auc"
}

model = lgb.train(params, 
                  train_set=trains, 
                  valid_sets=valids, 
                  num_boost_round=1000, early_stopping_rounds=100)

/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Number of positive: 2519, number of negative: 19183
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1243
[LightGBM] [Info] Number of data points in the train set: 21702, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116072 -> initscore=-2.030162
[LightGBM] [Info] Start training from score -2.030162
[1]	valid_0's auc: 0.873039
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.902138
[3]	valid_0's auc: 0.911537
[4]	valid_0's auc: 0.913536
[5]	valid_0's auc: 0.917607
[6]	valid_0's auc: 0.91991
[7]	valid_0's auc: 0.920735
[8]	valid_0's auc: 0.921292
[9]	valid_0's auc: 0.922296
[10]	valid_0's auc: 0.923189
[11]	valid_0's auc: 0.923843
[12]	valid_0's auc: 0.925061
[13]	valid_0's auc: 0.925297
[14]	valid_

In [93]:
testX = test.copy()

In [69]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18083 entries, 0 to 18082
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         18083 non-null  int64   
 1   age        18083 non-null  int64   
 2   job        18083 non-null  category
 3   marital    18083 non-null  category
 4   education  18083 non-null  category
 5   default    18083 non-null  category
 6   balance    18083 non-null  int64   
 7   housing    18083 non-null  category
 8   loan       18083 non-null  category
 9   contact    18083 non-null  category
 10  day        18083 non-null  int64   
 11  month      18083 non-null  category
 12  duration   18083 non-null  int64   
 13  campaign   18083 non-null  int64   
 14  pdays      18083 non-null  int64   
 15  previous   18083 non-null  int64   
 16  poutcome   18083 non-null  category
dtypes: category(9), int64(8)
memory usage: 1.3 MB


In [70]:
for col in ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]:
    testX[col] = test[col].astype("category")

In [71]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18083 entries, 0 to 18082
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         18083 non-null  int64   
 1   age        18083 non-null  int64   
 2   job        18083 non-null  category
 3   marital    18083 non-null  category
 4   education  18083 non-null  category
 5   default    18083 non-null  category
 6   balance    18083 non-null  int64   
 7   housing    18083 non-null  category
 8   loan       18083 non-null  category
 9   contact    18083 non-null  category
 10  day        18083 non-null  int64   
 11  month      18083 non-null  category
 12  duration   18083 non-null  int64   
 13  campaign   18083 non-null  int64   
 14  pdays      18083 non-null  int64   
 15  previous   18083 non-null  int64   
 16  poutcome   18083 non-null  category
dtypes: category(9), int64(8)
memory usage: 1.3 MB


In [72]:
pred = model.predict(testX)

In [73]:
pred

array([0.66701643, 0.6730471 , 0.01883297, ..., 0.00184338, 0.00644143,
       0.00728678])

In [74]:
sample[1] = pred

In [75]:
sample.to_csv("lgb_bank1.csv", index=None, header=None)

In [89]:
model.feature_importance(importance_type="split")

array([157, 170,  38,  33,   9,   7, 208,  57,  17,  85, 219, 191, 337,
        72, 142,  28,  30], dtype=int32)

In [90]:
pd.DataFrame(model.feature_importance(importance_type="split"), index=valX.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
duration,337
day,219
balance,208
month,191
age,170
id,157
pdays,142
contact,85
campaign,72
housing,57


In [91]:
model.feature_importance(importance_type="gain")

array([ 1022.34221077,  1695.52275681,   367.74752104,   294.51625681,
          56.40056038,    39.47429967,  1645.95513487,  1894.08394575,
         201.99473763,  2200.45195246,  2691.93933034,  5520.94137025,
       19106.71541071,   643.25726724,  2021.10009193,   234.95312953,
        4926.48470163])

In [92]:
pd.DataFrame(model.feature_importance(importance_type="gain"), index=valX.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
duration,19106.715411
month,5520.941370
poutcome,4926.484702
day,2691.939330
contact,2200.451952
pdays,2021.100092
housing,1894.083946
age,1695.522757
balance,1645.955135
id,1022.342211


In [102]:
train = train.drop("default", axis=1)

In [103]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27128 entries, 0 to 27127
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         27128 non-null  int64   
 1   age        27128 non-null  int64   
 2   job        27128 non-null  category
 3   marital    27128 non-null  category
 4   education  27128 non-null  category
 5   balance    27128 non-null  int64   
 6   housing    27128 non-null  category
 7   loan       27128 non-null  category
 8   contact    27128 non-null  category
 9   day        27128 non-null  int64   
 10  month      27128 non-null  category
 11  duration   27128 non-null  int64   
 12  campaign   27128 non-null  int64   
 13  pdays      27128 non-null  int64   
 14  previous   27128 non-null  int64   
 15  poutcome   27128 non-null  category
 16  y          27128 non-null  int64   
dtypes: category(8), int64(9)
memory usage: 2.1 MB


In [104]:
testX = testX.drop("default", axis=1)

In [105]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18083 entries, 0 to 18082
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         18083 non-null  int64   
 1   age        18083 non-null  int64   
 2   job        18083 non-null  category
 3   marital    18083 non-null  category
 4   education  18083 non-null  category
 5   balance    18083 non-null  int64   
 6   housing    18083 non-null  category
 7   loan       18083 non-null  category
 8   contact    18083 non-null  category
 9   day        18083 non-null  int64   
 10  month      18083 non-null  category
 11  duration   18083 non-null  int64   
 12  campaign   18083 non-null  int64   
 13  pdays      18083 non-null  int64   
 14  previous   18083 non-null  int64   
 15  poutcome   18083 non-null  category
dtypes: category(8), int64(8)
memory usage: 1.2 MB


In [106]:
df_train, df_val = train_test_split(train, test_size=0.2, random_state=0)

col = "y"
trainy = df_train[col]
trainX = df_train.drop(col, axis=1)

valy = df_val[col]
valX = df_val.drop(col, axis=1)

trains = lgb.Dataset(trainX, trainy)
valids = lgb.Dataset(valX, valy)

params = {
    "objective":"binary",
    "metrics":"auc"
}

model = lgb.train(params, 
                  train_set=trains, 
                  valid_sets=valids, 
                  num_boost_round=1000, early_stopping_rounds=100)

/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Number of positive: 2519, number of negative: 19183
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001848 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1240
[LightGBM] [Info] Number of data points in the train set: 21702, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116072 -> initscore=-2.030162
[LightGBM] [Info] Start training from score -2.030162
[1]	valid_0's auc: 0.873039
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.902138
[3]	valid_0's auc: 0.911537
[4]	valid_0's auc: 0.913536
[5]	valid_0's auc: 0.917607
[6]	valid_0's auc: 0.91991
[7]	valid_0's auc: 0.920735
[8]	valid_0's auc: 0.921292
[9]	valid_0's auc: 0.922296
[10]	valid_0's auc: 0.923189
[11]	valid_0's auc: 0.923843
[12]	valid_0's auc: 0.925061
[13]	valid_0's auc: 0.925297
[14]	valid_

In [107]:
pred = model.predict(testX)

In [108]:
pred

array([7.15911943e-01, 6.93213015e-01, 1.30775326e-02, ...,
       4.83200445e-04, 1.77595469e-03, 2.58648834e-03])

In [113]:
sample[1] = pred

In [115]:
sample.to_csv("lgb_bank2.csv", index=None, header=None)

In [109]:
model.feature_importance(importance_type="split")

array([503, 439,  89, 103,  20, 541,  93,  32, 137, 431, 279, 687, 150,
       303,  82,  41], dtype=int32)

In [112]:
pd.DataFrame(model.feature_importance(importance_type="split"), index=valX.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
duration,687
balance,541
id,503
age,439
day,431
pdays,303
month,279
campaign,150
contact,137
marital,103


In [116]:
model.feature_importance(importance_type="gain")

array([ 2269.82490456,  2699.82036757,   573.54376644,   574.44035673,
          81.51354778,  2819.59841627,  2061.27269745,   253.02936947,
        2400.40937978,  3512.5025984 ,  5889.70203012, 20285.11684394,
         838.45460325,  2511.42359364,   430.26009333,  4945.64135081])

In [118]:
pd.DataFrame(model.feature_importance(importance_type="gain"), index=valX.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
duration,20285.116844
month,5889.702030
poutcome,4945.641351
day,3512.502598
balance,2819.598416
age,2699.820368
pdays,2511.423594
contact,2400.409380
id,2269.824905
housing,2061.272697


In [119]:
train = train.drop("education", axis=1)

In [120]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27128 entries, 0 to 27127
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        27128 non-null  int64   
 1   age       27128 non-null  int64   
 2   job       27128 non-null  category
 3   marital   27128 non-null  category
 4   balance   27128 non-null  int64   
 5   housing   27128 non-null  category
 6   loan      27128 non-null  category
 7   contact   27128 non-null  category
 8   day       27128 non-null  int64   
 9   month     27128 non-null  category
 10  duration  27128 non-null  int64   
 11  campaign  27128 non-null  int64   
 12  pdays     27128 non-null  int64   
 13  previous  27128 non-null  int64   
 14  poutcome  27128 non-null  category
 15  y         27128 non-null  int64   
dtypes: category(7), int64(9)
memory usage: 2.0 MB


In [121]:
testX = testX.drop("education", axis=1)

In [122]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18083 entries, 0 to 18082
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        18083 non-null  int64   
 1   age       18083 non-null  int64   
 2   job       18083 non-null  category
 3   marital   18083 non-null  category
 4   balance   18083 non-null  int64   
 5   housing   18083 non-null  category
 6   loan      18083 non-null  category
 7   contact   18083 non-null  category
 8   day       18083 non-null  int64   
 9   month     18083 non-null  category
 10  duration  18083 non-null  int64   
 11  campaign  18083 non-null  int64   
 12  pdays     18083 non-null  int64   
 13  previous  18083 non-null  int64   
 14  poutcome  18083 non-null  category
dtypes: category(7), int64(8)
memory usage: 1.2 MB


In [123]:
df_train, df_val = train_test_split(train, test_size=0.2, random_state=0)

col = "y"
trainy = df_train[col]
trainX = df_train.drop(col, axis=1)

valy = df_val[col]
valX = df_val.drop(col, axis=1)

trains = lgb.Dataset(trainX, trainy)
valids = lgb.Dataset(valX, valy)

params = {
    "objective":"binary",
    "metrics":"auc"
}

model = lgb.train(params, 
                  train_set=trains, 
                  valid_sets=valids, 
                  num_boost_round=1000, early_stopping_rounds=100)

/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/kumazawateruhito/opt/anaconda3/envs/base_root/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Number of positive: 2519, number of negative: 19183
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002736 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1235
[LightGBM] [Info] Number of data points in the train set: 21702, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116072 -> initscore=-2.030162
[LightGBM] [Info] Start training from score -2.030162
[1]	valid_0's auc: 0.873039
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.902138
[3]	valid_0's auc: 0.911537
[4]	valid_0's auc: 0.913536
[5]	valid_0's auc: 0.917607
[6]	valid_0's auc: 0.91991
[7]	valid_0's auc: 0.920735
[8]	valid_0's auc: 0.921292
[9]	valid_0's auc: 0.922296
[10]	valid_0's auc: 0.923189
[11]	valid_0's auc: 0.923843
[12]	valid_0's auc: 0.925061
[13]	valid_0's auc: 0.925297
[14]	valid_

In [124]:
pred = model.predict(testX)

In [125]:
pred

array([0.64339752, 0.65165109, 0.00859225, ..., 0.00218701, 0.01064278,
       0.00593805])

In [126]:
sample[1] = pred

In [127]:
sample.to_csv("lgb_bank3.csv", index=None, header=None)

In [128]:
model.feature_importance(importance_type="split")

array([117, 147,  45,  42, 177,  60,  19,  75, 188, 196, 331,  47, 125,
        20,  31], dtype=int32)

In [133]:
pd.DataFrame(model.feature_importance(importance_type="split"), index=valX.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
duration,331
month,196
day,188
balance,177
age,147
pdays,125
id,117
contact,75
housing,60
campaign,47


In [134]:
model.feature_importance(importance_type="gain")

array([  804.39894962,  1600.72898388,   428.1671803 ,   365.1719389 ,
        1466.6315515 ,  1911.11407423,   211.23256707,  2140.07857323,
        2553.70424008,  5556.22169685, 19104.60957837,   467.91748714,
        1871.1838913 ,   169.88038874,  4920.05955744])

In [135]:
pd.DataFrame(model.feature_importance(importance_type="gain"), index=valX.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
duration,19104.609578
month,5556.221697
poutcome,4920.059557
day,2553.704240
contact,2140.078573
housing,1911.114074
pdays,1871.183891
age,1600.728984
balance,1466.631552
id,804.398950
